Przed oddaniem zadania upewnij się, że wszystko działa poprawnie.
**Uruchom ponownie kernel** (z paska menu: Kernel$\rightarrow$Restart) a następnie
**wykonaj wszystkie komórki** (z paska menu: Cell$\rightarrow$Run All).

Upewnij się, że wypełniłeś wszystkie pola `TU WPISZ KOD` lub `TU WPISZ ODPOWIEDŹ`, oraz
że podałeś swoje imię i nazwisko poniżej:

In [1]:
NAME = "Maciej Wilhelmi"

---

# 1. Wprowadzenie do przetwarzania grafów.

Na ostatnim wykładzie (Wykład 3) omówiliśmy blueprint geometrycznego uczenia głębokiego oraz różne dziedziny danych, w tym dziedzinę zbiorów i grafów. Przez najbliższe dwa wykłady i laboratoria skupimy się na temacie przetwarzania grafów i uczenia reprezentacji na nich.

Grafy opisują obiekty i relacje między obiektami. Możemy je spotkać w wielu dziedzinach, takich jak analiza mediów społecznościowych (użytkownicy portali i interakcje między nimi), chemii obliczeniowej (cząsteczki jako atomy połączone za pomocą wiązań atomowych) czy astronomii (relacje grawitacyjne między ciałami niebieskimi).

Grafy nie muszą koniecznie opisywać danych, które mają bezpośrednio strukturę sieciową. Możemy również zastosować metody przetwarzania grafów do opisu tradycyjnych typów danych, np. dla zbioru obrazów możemy obliczyć ich podobieństwa i jeśli to podobieństwo jest większe niż zadana wartość progowa, to tworzymy między takimi obrazami krawędź; jeśli na obrazie znajduje się kilka obiektów, możemy opisać ich względne położenia za pomocą krawędzi o różnych typach (np. "X znajduje się z lewej strony Y", "X przesłania część Y" itd.).


W trakcie obecnego laboratorium, które obejmuje dwa kolejne zajęcia (Laboratorium 3 oraz 4):
- zapoznamy się z biblioteką Pytorch-Geometric,
- poznamy wybrane tradycyjne metody uczenia reprezentacji dla grafów,
- nauczymy się przygotowywać dane i ewaluować modele w 3 popularnych zadaniach grafowych,
- poznamy wybrane architektury grafowych sieci neuronowych,
- we wszystkich powyższych zagadnieniach spróbujemy zbadać wpływ hiperparametrów metod na ich jakość działania.

## 1.1. Graf

Grafem $\mathcal{G}$ nazywamy krotkę $\mathcal{G} = (\mathcal{V}, \mathcal{E})$, gdzie $\mathcal{V}$ to zbiór wierzchołków, a $\mathcal{E} \in \mathcal{V} \times \mathcal{V}$ to zbiór krawędzi łączących pary wierzchołków. 

Z każdym wierzchołkiem $u$ skojarzony jest wektor atrybutów $\mathbf{x}_u \in \mathbb{R}^{d}$. Cechy wszystkich wierzchołków są opisane przez macierz $\mathbf{X} \in \mathbb{R}^{|\mathcal{V}| \times d}$. Podobnie można zdefiniować atrybuty krawędzi oraz atrybuty całego grafu, przy czym dla uproszczenia przyjmiemy, że tylko wierzchołki posiadają atrybuty.

W celu opisania połączeń w grafie stosuje się najczęściej (binarną) macierz sąsiedztwa $A \in \{0, 1\}^{|\mathcal{V}| \times |\mathcal{V}|}$, w której: $a_{uv} = 1 \iff (u, v) \in \mathcal{E}$. Macierz ta jest macierzą symetryczną dla grafów nieskierowanych (krawędź nie ma kierunku), natomiast w przypadku grafów skierowanych raczej jest macierzą niesymetryczną.

Innym sposobem opisu połączeń jest lista krawędzi (do której jeszcze wrócimy za chwilę). Motywacją do stosowania takiego zapisu jest fakt, że najczęściej graf jest rzadki, tzn. liczba krawędzi jest znacząco mniejsza niż maksymalna liczba możliwych krawędzi -- $|\mathcal{E}| \ll |\mathcal{V}|  \times |\mathcal{V}|$.

## 1.2. PyTorch-Geometric

Bibliotek wykorzystywanych do uczenia modeli działających na danych grafowych jest wiele, jednak dwie z nich są bardzo często stosowane: **PyTorch-Geometric** (od niedawna zwany **PyG**; [dokumentacja](https://pytorch-geometric.readthedocs.io/en/latest/)) oraz **Deep Graph Library** (w skrócie DGL; [dokumentacja](https://docs.dgl.ai)). W ramach laboratorium będzie wykorzystywać bibliotekę PyTorch-Geometric, jednak zachęcamy zainteresowanych do zapoznania się również z DGL.

PyG zawiera wiele gotowych do użycia: (a) zbiorów danych (grafy jednorelacyjne i heterogeniczne, małe i wielkoskalowe, zbiory z wieloma grafami itp.), (b) warstw stosowanych w grafowych sieciach neuronowych (GNN) oraz (c) modeli sieci neuronowych do przetwarzania grafów.

Podstawową strukturą danych opisującą grafy w PyG jest obiekt `Data`. Zawiera on opis całego grafu w następującej postaci:
- atrybuty wierzchołków - tensor `data.x`
- listę krawędzi - tensor `data.edge_index` w formacie COO (o wymiarach $2 \times |\mathcal{E}|$)
- (opcjonalnie) klasy wierzchołków / krawędzi / grafu - tensor `data.y` (wymiar zależny od problemu)
- (opcjonalnie) atrybuty krawędzi - tensor `data.edge_attr`
- pola zawierające informacje o liczbie krawędzi, liczbie wierzchołków oraz wymiarowościach ich atrybutów,
- inne pola, które pomijamy w trakcie tego laboratorium.

W przypadku problemów, gdzie działamy na zbiorze składającym się z wielu grafów, każdy z nich jest opisany przez osobny obiekt `Data`. 

Najczęściej, w przypadku małych i średniej wielkości grafów, stosowany jest scenariusz, w którym cały graf jest przetwarzany na raz (ang. *full-batch*). Używanie klasy `DataLoader`, która jest często stosowana w innych dziedzinach uczenia głębokiego, mija się tutaj z celem, ponieważ mamy tylko jeden graf. Będziemy zatem przekazywać do modelu bezpośrednio obiekt `Data`. Wyjątkiem będzie scenariusz z wieloma grafami.

## 1.3. Przykład
Zacznijmy od załadowania zbioru `Cora`. Jest to zbiór, w którym wierzchołki opisują publikacje naukowe, natomiast krawędzie opisują cytowania między artykułami. Każdy wierzchołek jest opisany przez binarny wektor worka słów użytych w artykule (ang. *binary bag-of-words*). Każdy artykuł / wierzchołek posiada przypisaną klasę. Zbiór ten można zastosować w tzw. problemie klasyfikacji wierzchołków lub problemie predykcji połączeń. Obecnie zbiór nie jest już stosowany do porównywania metod uczenia reprezentacji na grafach, głównie ze względu na wielkość (mały zbiór) oraz wysoką homofilię (więcej o tym na wykładzie).

In [ ]:
# !pip install torch_geometric

In [1]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root="./data/", name="Cora")

print(f"Liczba grafów: {len(dataset)}")

data = dataset[0]

Liczba grafów: 1


## Zadanie 1.1. (1.5 pkt)
Zaimplementuj funkcję `print_statistics`, która dla podanego obiektu `Data` wypisze następujące informacje / statystyki o grafie:
- liczba wierzchołków
- liczba krawędzi
- wymiarowość atrybutów wierzchołków
- liczba klas wierzchołków
- czy graf jest skierowany
- gęstość grafu (tzn. liczba krawędzi do maksymalnej możliwej liczby krawędzi; w procentach)

In [4]:
from torch_geometric.data import Data
import torch_geometric.utils
import torch


def print_statistics(data: Data) -> None:
    # TU WPISZ KOD
    num_nodes = data.num_nodes
    num_edges = data.num_edges
    num_node_features = data.num_node_features
    num_classes = len(torch.unique(data.y))
    is_directed = data.is_directed()

    max_edges = (num_nodes * (num_nodes - 1)) / 2
    density = 100 * num_edges / max_edges


    print(f'Liczba wierzchołków: {num_nodes}\nLiczba krawędzi: {num_edges}\
    \nWymiarowość atrybutów wierzchołków: {num_node_features}\nLiczba klas wierzchołków: {num_classes}\
    \nSkierowany? {is_directed}\ngęstość: {density}%')
        
    
    
print_statistics(data=data)

Liczba wierzchołków: 2708
Liczba krawędzi: 10556    
Wymiarowość atrybutów wierzchołków: 1433
Liczba klas wierzchołków: 7    
Skierowany? False
gęstość: 0.2879999825388415%
